In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime

In [5]:
# Other Options & Parameters -------------------------------------------------------------------
pd.set_option('mode.chained_assignment',None)
 


# Functions -------------------------------------------------------------------------------------
def download_vix_and_spy_data():
    d = datetime.timedelta(days=120)
    end_date = datetime.date.today()
    start_date = end_date - d
    spx_df = yf.download("^GSPC", start=start_date)
    spx_df['Return'] = (spx_df['Close'] - spx_df['Close'].shift(1)) / spx_df['Close'].shift(1)
    spx_df = spx_df[['Close','Return']]
    spx_df['Realised Volatility'] = spx_df['Return'].rolling(21).std()*np.sqrt(252)*100
 
    vix_df = yf.download("^VIX", start=start_date)
    vix_df['VIX'] = vix_df['Close']
    vix_df = vix_df['VIX']
 
    df = spx_df.merge(vix_df, left_index=True, right_index=True, how='inner')
    df['VRP'] = df['VIX'].shift(21) - df['Realised Volatility']
 
    return df

In [6]:
df = download_vix_and_spy_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [16]:
from ib_insync import *

util.startLoop()
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1) 

<IB connected to 127.0.0.1:7497 clientId=1>

In [26]:
contract = Index('VIX', 'CBOE')


In [27]:
ib.qualifyContracts(contract)

[Index(conId=13455763, symbol='VIX', exchange='CBOE', currency='USD', localSymbol='VIX')]

In [32]:
ib.reqContractDetails(contract)

[ContractDetails(contract=Contract(secType='IND', conId=13455763, symbol='VIX', exchange='CBOE', currency='USD', localSymbol='VIX'), marketName='', minTick=0.01, orderTypes='ACTIVETIM,AD,ADJUST,ALERT,ALLOC,BASKET,BENCHPX,COND,CONDORDER,DAY,DEACT,DEACTDIS,DEACTEOD,GAT,GTC,GTD,GTT,HID,LMT,NONALGO,OCA,SCALE,SCALERST,WHATIF', validExchanges='CBOE', priceMagnifier=1, underConId=0, longName='CBOE Volatility Index', contractMonth='', industry='Indices', category='Volatility Index', subcategory='*', timeZoneId='US/Central', tradingHours='20231204:0215-20231204:0815;20231204:0830-20231204:1600;20231205:0215-20231205:0815;20231205:0830-20231205:1600;20231206:0215-20231206:0815;20231206:0830-20231206:1600;20231207:0215-20231207:0815;20231207:0830-20231207:1600;20231208:0215-20231208:0815;20231208:0830-20231208:1600', liquidHours='20231204:0215-20231204:0815;20231204:0830-20231204:1600;20231205:0215-20231205:0815;20231205:0830-20231205:1600;20231206:0215-20231206:0815;20231206:0830-20231206:1600;2

In [63]:
def get_vxm_term_structure():
    # Get today's date
    today = datetime.datetime.now()

    # Dictionary to store futures data
    futures_data = {
        'Contract': [],
        'LastPrice': [],
        'DTE': [],
        'AnnualizedYield': []
    }

    # Set market data type to delayed frozen data
    ib.reqMarketDataType(4)

    # Get the spot rate from your function
    spot_rate_df = download_vix_and_spy_data()
    spot_rate = spot_rate_df['VIX'].iloc[-1]  # Latest VIX spot rate

    for i in range(9):  # Next 9 maturities
        # Calculate the contract month
        month = (today.month + i - 1) % 12 + 1
        year = today.year + (today.month + i - 1) // 12
        contract_month = f"{year}{month:02}"

        # Find the futures contract
        fut = ib.qualifyContracts(Future('VXM', lastTradeDateOrContractMonth=contract_month))

        if fut:
            # Fetch the latest market data
            market_data = ib.reqMktData(fut[0])
            ib.sleep(1)  # Wait for the data to be fetched
            last_price = market_data.last

            # Ensure that we have valid market data
            if last_price is not None:
                # Calculate days until expiration
                expiration_date = datetime.datetime.strptime(fut[0].lastTradeDateOrContractMonth, '%Y%m%d')
                dte = (expiration_date - today).days

                # Calculate annualized yield
                annualized_yield = ((spot_rate / last_price) - 1) * (365 / dte)

                # Append the data to the dictionary
                futures_data['Contract'].append(fut[0].localSymbol)
                futures_data['LastPrice'].append(last_price)
                futures_data['DTE'].append(dte)
                futures_data['AnnualizedYield'].append(annualized_yield)

    # Convert the dictionary to DataFrame
    futures_df = pd.DataFrame(futures_data)
    return futures_df

In [64]:
vxm_term_structure = get_vxm_term_structure()
vxm_term_structure

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Error 200, reqId 59: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Future(symbol='VXM', lastTradeDateOrContractMonth='202406')
Unknown contract: Future(symbol='VXM', lastTradeDateOrContractMonth='202406')
Error 200, reqId 60: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Future(symbol='VXM', lastTradeDateOrContractMonth='202407')
Unknown contract: Future(symbol='VXM', lastTradeDateOrContractMonth='202407')
Error 200, reqId 61: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Future(symbol='VXM', lastTradeDateOrContractMonth='202408')
Unknown contract: Future(symbol='VXM', lastTradeDateOrContractMonth='202408')


,Contract,LastPrice,DTE,AnnualizedYield
0,VXMZ3,13.84,15,-1.336224
1,VXMF4,15.55,43,-1.348314
2,VXMG4,16.50,71,-1.065557
3,VXMH4,17.10,106,-0.809500
4,VXMJ4,17.60,134,-0.699542
5,VXMK4,18.00,169,-0.590335


In [74]:
def choose_future_to_short(futures_data):
    """
    Determine which VIX future to short based on the annualized yield and specific criteria.

    Args:
    futures_data (DataFrame): DataFrame containing futures contract data.

    Returns:
    str: The contract to short.
    """

    # Sort the DataFrame by the absolute value of AnnualizedYield in ascending order
    futures_data['AbsYield'] = futures_data['AnnualizedYield'].abs()
    sorted_futures = futures_data.sort_values(by='AbsYield')

    # Iterate through the sorted DataFrame
    for i in range(len(sorted_futures) - 1):
        current_future = sorted_futures.iloc[i]
        next_future = sorted_futures.iloc[i + 1]

        # Check if current future yield is equal to or slightly greater than the next
        # and if the current price is below 16
        if current_future['AbsYield'] >= next_future['AbsYield'] * 0.98 and current_future['LastPrice'] < 16:
            # Choose the next month's future
            return next_future['Contract']

    # If no future meets the criteria, choose the one with the lowest absolute yield
    return sorted_futures.iloc[0]['Contract']

choose_future_to_short(vxm_term_structure)

'VXMF4'

In [75]:
# Check for market structure - Contango or Backwardation
is_contango = vxm_term_structure['LastPrice'].is_monotonic_increasing
is_contango

True

In [69]:
def calculate_number_of_contracts(allocated_amount, contract_price, contract_size):
    """
    Calculate the number of futures contracts to trade based on allocated amount.

    Args:
    allocated_amount (float): The dollar amount allocated for the futures contracts.
    contract_price (float): The price of a single futures contract.
    contract_size (float): The size of a single futures contract.

    Returns:
    int: The number of futures contracts to trade.
    """
    total_value = contract_price * contract_size
    number_of_contracts = allocated_amount // total_value
    return int(number_of_contracts)

# Example usage
allocated_amount = 0.07*57000  # $100,000 allocated for trading
contract_price = 15.55  # Example futures contract price
contract_size = 100  # This is an example size, you need to replace it with the actual contract size

num_contracts = calculate_number_of_contracts(allocated_amount, contract_price, contract_size)
print(f"Number of contracts to trade: {num_contracts}")


Number of contracts to trade: 2


In [38]:

today = datetime.datetime.now()

# DataFrame to store futures data
futures_data = pd.DataFrame()

for i in range(9):  # Next 9 maturities
    # Find the futures contract
    print(f"{today.year + (today.month + i - 1) // 12}{(today.month + i - 1) % 12 + 1:02}")

202312
202401
202402
202403
202404
202405
202406
202407
202408


In [40]:
ib.reqMarketDataType(4)
market_data = ib.reqMktData(fut[0]).last

In [42]:
market_data.last

15.55

In [43]:
spot_rate_df = download_vix_and_spy_data()
spot_rate_df


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Close,Return,Realised Volatility,VIX,VRP
Date,,,,,
2023-08-07,4518.439941,NaN,NaN,15.77,NaN
2023-08-08,4499.379883,-0.004218,NaN,15.99,NaN
2023-08-09,4467.709961,-0.007039,NaN,15.96,NaN
2023-08-10,4468.830078,0.000251,NaN,15.85,NaN
2023-08-11,4464.049805,-0.001070,NaN,14.84,NaN
...,...,...,...,...,...
2023-11-28,4554.890137,0.000980,11.333491,12.69,9.936510
2023-11-29,4550.580078,-0.000946,11.189135,12.98,8.560865
2023-11-30,4567.799805,0.003784,11.159825,12.92,6.980174


In [44]:
spot_rate = spot_rate_df['VIX'].iloc[-1]  
spot_rate

13.050000190734863